In [22]:
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from pandas import Categorical
from pandas import crosstab
from pandas import merge
from pandas import to_datetime
from pandas import set_option
import time
import calendar
from random import randint
from geopy.geocoders import Nominatim #https://geopy.readthedocs.io/en/stable/

In [23]:
df = read_csv('train.csv', delimiter = ',')
    
def data_old(df):
    
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def data_analytics(df):
    
    df_null = (DataFrame(df.isnull().sum(),
          columns = ['quantities_number_null'])
         .rename_axis('Columns'))
    df_null.reset_index(level=0, inplace=True)
    
    df_missing = (DataFrame((df.isnull().sum() / len(df))*100,
         columns = ['percentage_data_missing'])
         .rename_axis('Columns'))
    df_missing.reset_index(level=0, inplace=True)
    
    df_type = (DataFrame(df.dtypes,
          columns = ['data_type'])
         .rename_axis('Columns'))
    df_type.reset_index(level=0, inplace=True)

    df_1 = merge(df_null, df_missing)
    df = merge(df_1, df_type)
    return df

def data_new(df):
    
    if 'store_and_fwd_flag' in df.columns and 'id' in df.columns: 
        df = df.drop(columns=['store_and_fwd_flag'])
        df = df.drop(columns=['id'])
        
    if 'vendor_id' in df.columns:
        df['vendor_id'] = (df['vendor_id']
                           .map({1:'class_one',
                                 2:'class_two'}))
        df['vendor_id'] = df['vendor_id'].astype('category')
        
    if 'pickup_datetime' in df.columns and 'dropoff_datetime' in df.columns:
        df['pickup_datetime'] = to_datetime(df['pickup_datetime'])
        df['dropoff_datetime'] = to_datetime(df['dropoff_datetime'])
    
    if 'pickup_longitude' in df.columns and 'pickup_latitude' in df.columns:
        df['pickup_longitude'] = df['pickup_longitude'].astype(str, errors = 'raise')
        df['pickup_latitude'] = df['pickup_latitude'].astype(str, errors = 'raise')
        df["pickup_lat_lon"] = df['pickup_latitude'].map(str) + "," + df['pickup_longitude'].map(str)
        df = df.drop(columns=['pickup_longitude', 'pickup_latitude'])
        
    if 'dropoff_longitude' in df.columns and 'dropoff_latitude' in df.columns:
        df['dropoff_longitude'] = df['dropoff_longitude'].astype(str, errors = 'raise')
        df['dropoff_latitude'] = df['dropoff_latitude'].astype(str, errors = 'raise')
        df["dropoff_lat_lon"] = df['dropoff_latitude'].map(str) + "," + df['dropoff_longitude'].map(str)
        df = df.drop(columns=['dropoff_longitude', 'dropoff_latitude'])
    
    if 'trip_duration' in df.columns:
        df['trip_duration'] = df['dropoff_datetime'] - df['pickup_datetime'] 
        
    if 'dropoff_datetime' in df.columns and 'pickup_datetime' in df.columns:
        df['pickup_date'] = to_datetime(df['pickup_datetime'].dt.date)
        df['pickup_day'] = df['pickup_datetime'].apply(lambda x : x.day)
        df['pickup_hour'] = df['pickup_datetime'].apply(lambda x : x.hour)
        df['pickup_day_of_week'] = df['pickup_datetime'].apply(lambda x : calendar.day_name[x.weekday()])
        df['dropoff_date'] = to_datetime(df['dropoff_datetime'].dt.date)
        df['dropoff_day'] = df['dropoff_datetime'].apply(lambda x : x.day)
        df['dropoff_hour'] = df['dropoff_datetime'].apply(lambda x : x.hour)
        df['dropoff_day_of_week'] = df['dropoff_datetime'].apply(lambda x : calendar.day_name[x.weekday()])
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def data_address(df):
    df = df.sample(n=10)
    
    geolocator = Nominatim(user_agent="myGeocoder")
    
    if 'pickup_lat_lon' in df.columns and 'dropoff_lat_lon' in df.columns:
        df['pickup_address'] = df.apply(lambda row: geolocator.reverse(row['pickup_lat_lon']).raw,axis=1)
        df['dropoff_address'] = df.apply(lambda row: geolocator.reverse(row['dropoff_lat_lon']).raw,axis=1)
    
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def dictionary(df):
    
    data = (df
           .reset_index(drop=True)['pickup_address']
           .to_dict())
    
    data = data[randint(0,9)]
    
    for chave in data.keys():
        print(f'Chave = {chave} e Valor = {data[chave]}')
        
def final_step_with_api(df):
    
    if 'pickup_lat_lon' in df.columns and 'dropoff_lat_lon' in df.columns:
        df['pickup_country'] = (df['pickup_address']
                                .apply(lambda row: row['address']['country']))
        df['dropoff_country'] = (df['dropoff_address']
                                .apply(lambda row: row['address']['country']))
        df['pickup_state'] = (df['pickup_address']
                                .apply(lambda row: row['address']['state']))
        df['dropoff_state'] = (df['dropoff_address']
                                .apply(lambda row: row['address']['state']))
        
    df.drop(columns=['pickup_address','dropoff_address'], axis=1, inplace=True)
    
    df = df[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 
             'passenger_count', 'trip_duration', 'pickup_state', 
             'pickup_country', 'dropoff_state', 'dropoff_country',
             'pickup_date', 'pickup_day', 'pickup_hour', 'pickup_day_of_week',
             'dropoff_date', 'dropoff_day', 'dropoff_hour', 'dropoff_day_of_week',
             'pickup_lat_lon', 'dropoff_lat_lon']]
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def final_step(df):
    
    df = df[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 
             'passenger_count', 'trip_duration', 'pickup_date', 
             'pickup_day', 'pickup_hour', 'pickup_day_of_week',
             'dropoff_date', 'dropoff_day', 'dropoff_hour', 'dropoff_day_of_week',
             'pickup_lat_lon', 'dropoff_lat_lon']]
    
    df.to_csv('new_base_taxi.csv')
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

In [24]:
# 1º 
data_old(df.head())

O dataset contém 5 amostras e 11 colunas


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [25]:
# 2º
data_analytics(df)

,Columns,quantities_number_null,percentage_data_missing,data_type
0,id,0,0.0,object
1,vendor_id,0,0.0,int64
2,pickup_datetime,0,0.0,object
3,dropoff_datetime,0,0.0,object
4,passenger_count,0,0.0,int64
5,pickup_longitude,0,0.0,float64
6,pickup_latitude,0,0.0,float64
7,dropoff_longitude,0,0.0,float64
8,dropoff_latitude,0,0.0,float64
9,store_and_fwd_flag,0,0.0,object


In [26]:
# 3º
dt = data_new(df)
display(dt.head(3))
display(data_analytics(dt))

O dataset contém 1458644 amostras e 15 colunas


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_duration,pickup_lat_lon,dropoff_lat_lon,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week
0,class_two,2016-03-14 17:24:55,2016-03-14 17:32:30,1,00:07:35,"40.76793670654297,-73.98215484619139","40.765602111816406,-73.96463012695312",2016-03-14,14,17,Monday,2016-03-14,14,17,Monday
1,class_one,2016-06-12 00:43:35,2016-06-12 00:54:38,1,00:11:03,"40.738563537597656,-73.98041534423827","40.73115158081055,-73.99948120117188",2016-06-12,12,0,Sunday,2016-06-12,12,0,Sunday
2,class_two,2016-01-19 11:35:24,2016-01-19 12:10:48,1,00:35:24,"40.763938903808594,-73.97902679443358","40.71008682250977,-74.00533294677734",2016-01-19,19,11,Tuesday,2016-01-19,19,12,Tuesday


,Columns,quantities_number_null,percentage_data_missing,data_type
0,vendor_id,0,0.0,category
1,pickup_datetime,0,0.0,datetime64[ns]
2,dropoff_datetime,0,0.0,datetime64[ns]
3,passenger_count,0,0.0,int64
4,trip_duration,0,0.0,timedelta64[ns]
5,pickup_lat_lon,0,0.0,object
6,dropoff_lat_lon,0,0.0,object
7,pickup_date,0,0.0,datetime64[ns]
8,pickup_day,0,0.0,int64
9,pickup_hour,0,0.0,int64


In [27]:
# 4º
dt_api = data_address(dt)
dt_api.head()

O dataset contém 10 amostras e 17 colunas


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_duration,pickup_lat_lon,dropoff_lat_lon,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week,pickup_address,dropoff_address
284569,class_one,2016-03-15 13:08:47,2016-03-15 13:14:54,1,00:06:07,"40.760986328125,-73.9611587524414","40.77036666870117,-73.95597839355469",2016-03-15,15,13,Tuesday,2016-03-15,15,13,Tuesday,"{'place_id': 26078637, 'licence': 'Data © Open...","{'place_id': 166772262, 'licence': 'Data © Ope..."
343987,class_two,2016-02-02 19:47:33,2016-02-02 19:55:04,2,00:07:31,"40.755130767822266,-73.97756958007811","40.74623107910156,-73.97142028808595",2016-02-02,2,19,Tuesday,2016-02-02,2,19,Tuesday,"{'place_id': 25337275, 'licence': 'Data © Open...","{'place_id': 170276377, 'licence': 'Data © Ope..."
235724,class_two,2016-05-13 13:26:43,2016-05-13 13:29:09,2,00:02:26,"40.74905776977539,-73.99212646484375","40.74176788330078,-73.99034881591797",2016-05-13,13,13,Friday,2016-05-13,13,13,Friday,"{'place_id': 21005404, 'licence': 'Data © Open...","{'place_id': 168437608, 'licence': 'Data © Ope..."
422258,class_two,2016-03-05 08:05:36,2016-03-05 08:09:25,1,00:03:49,"40.79237747192383,-73.9679946899414","40.777156829833984,-73.98014831542969",2016-03-05,5,8,Saturday,2016-03-05,5,8,Saturday,"{'place_id': 16483726, 'licence': 'Data © Open...","{'place_id': 167601405, 'licence': 'Data © Ope..."
795066,class_one,2016-04-04 11:26:11,2016-04-04 11:35:37,1,00:09:26,"40.77365493774415,-73.95995330810547","40.76535034179688,-73.9612808227539",2016-04-04,4,11,Monday,2016-04-04,4,11,Monday,"{'place_id': 271485931, 'licence': 'Data © Ope...","{'place_id': 247769528, 'licence': 'Data © Ope..."


In [28]:
# 5º
dictionary(dt_api)

Chave = place_id e Valor = 21005404
Chave = licence e Valor = Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright
Chave = osm_type e Valor = node
Chave = osm_id e Valor = 2384821688
Chave = lat e Valor = 40.74907
Chave = lon e Valor = -73.992305
Chave = display_name e Valor = Astoria Bank, 370, 7th Avenue, Chelsea District, New York County, New York, 10001, United States
Chave = address e Valor = {'amenity': 'Astoria Bank', 'house_number': '370', 'road': '7th Avenue', 'neighbourhood': 'Chelsea District', 'county': 'New York County', 'city': 'New York', 'state': 'New York', 'ISO3166-2-lvl4': 'US-NY', 'postcode': '10001', 'country': 'United States', 'country_code': 'us'}
Chave = boundingbox e Valor = ['40.74902', '40.74912', '-73.992355', '-73.992255']


In [29]:
# 6º 
final_step_with_api(dt_api)

O dataset contém 10 amostras e 19 colunas


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_duration,pickup_state,pickup_country,dropoff_state,dropoff_country,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week,pickup_lat_lon,dropoff_lat_lon
284569,class_one,2016-03-15 13:08:47,2016-03-15 13:14:54,1,00:06:07,New York,United States,New York,United States,2016-03-15,15,13,Tuesday,2016-03-15,15,13,Tuesday,"40.760986328125,-73.9611587524414","40.77036666870117,-73.95597839355469"
343987,class_two,2016-02-02 19:47:33,2016-02-02 19:55:04,2,00:07:31,New York,United States,New York,United States,2016-02-02,2,19,Tuesday,2016-02-02,2,19,Tuesday,"40.755130767822266,-73.97756958007811","40.74623107910156,-73.97142028808595"
235724,class_two,2016-05-13 13:26:43,2016-05-13 13:29:09,2,00:02:26,New York,United States,New York,United States,2016-05-13,13,13,Friday,2016-05-13,13,13,Friday,"40.74905776977539,-73.99212646484375","40.74176788330078,-73.99034881591797"
422258,class_two,2016-03-05 08:05:36,2016-03-05 08:09:25,1,00:03:49,New York,United States,New York,United States,2016-03-05,5,8,Saturday,2016-03-05,5,8,Saturday,"40.79237747192383,-73.9679946899414","40.777156829833984,-73.98014831542969"
795066,class_one,2016-04-04 11:26:11,2016-04-04 11:35:37,1,00:09:26,New York,United States,New York,United States,2016-04-04,4,11,Monday,2016-04-04,4,11,Monday,"40.77365493774415,-73.95995330810547","40.76535034179688,-73.9612808227539"
1271434,class_two,2016-04-07 09:45:13,2016-04-07 09:54:13,2,00:09:00,New York,United States,New York,United States,2016-04-07,7,9,Thursday,2016-04-07,7,9,Thursday,"40.732460021972656,-73.9969711303711","40.73936080932617,-74.00887298583984"
1372074,class_two,2016-05-11 17:18:17,2016-05-11 17:33:03,5,00:14:46,New York,United States,New York,United States,2016-05-11,11,17,Wednesday,2016-05-11,11,17,Wednesday,"40.78084182739258,-73.97268676757811","40.762149810791016,-73.98301696777342"
465909,class_two,2016-03-27 18:46:41,2016-03-27 18:57:10,6,00:10:29,New York,United States,New York,United States,2016-03-27,27,18,Sunday,2016-03-27,27,18,Sunday,"40.71519088745117,-74.00553131103516","40.75414276123047,-73.98461151123048"
168483,class_one,2016-05-10 15:32:17,2016-05-10 15:39:43,1,00:07:26,New York,United States,New York,United States,2016-05-10,10,15,Tuesday,2016-05-10,10,15,Tuesday,"40.76483917236328,-73.98359680175781","40.775047302246094,-73.9847640991211"
1391623,class_one,2016-04-07 14:22:53,2016-04-07 14:33:42,1,00:10:49,New York,United States,New York,United States,2016-04-07,7,14,Thursday,2016-04-07,7,14,Thursday,"40.76424026489258,-73.95433807373048","40.766361236572266,-73.96747589111328"


In [30]:
final_step(dt)

O dataset contém 1458644 amostras e 15 colunas


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_duration,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week,pickup_lat_lon,dropoff_lat_lon
0,class_two,2016-03-14 17:24:55,2016-03-14 17:32:30,1,00:07:35,2016-03-14,14,17,Monday,2016-03-14,14,17,Monday,"40.76793670654297,-73.98215484619139","40.765602111816406,-73.96463012695312"
1,class_one,2016-06-12 00:43:35,2016-06-12 00:54:38,1,00:11:03,2016-06-12,12,0,Sunday,2016-06-12,12,0,Sunday,"40.738563537597656,-73.98041534423827","40.73115158081055,-73.99948120117188"
2,class_two,2016-01-19 11:35:24,2016-01-19 12:10:48,1,00:35:24,2016-01-19,19,11,Tuesday,2016-01-19,19,12,Tuesday,"40.763938903808594,-73.97902679443358","40.71008682250977,-74.00533294677734"
3,class_two,2016-04-06 19:32:31,2016-04-06 19:39:40,1,00:07:09,2016-04-06,6,19,Wednesday,2016-04-06,6,19,Wednesday,"40.719970703125,-74.01004028320312","40.70671844482422,-74.01226806640625"
4,class_two,2016-03-26 13:30:55,2016-03-26 13:38:10,1,00:07:15,2016-03-26,26,13,Saturday,2016-03-26,26,13,Saturday,"40.79320907592773,-73.97305297851562","40.78252029418945,-73.9729232788086"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,class_two,2016-04-08 13:31:04,2016-04-08 13:44:02,4,00:12:58,2016-04-08,8,13,Friday,2016-04-08,8,13,Friday,"40.745521545410156,-73.9822006225586","40.74016952514648,-73.99491119384766"
1458640,class_one,2016-01-10 07:35:15,2016-01-10 07:46:10,1,00:10:55,2016-01-10,10,7,Sunday,2016-01-10,10,7,Sunday,"40.74737930297852,-74.00094604492188","40.796546936035156,-73.97018432617188"
1458641,class_two,2016-04-22 06:57:41,2016-04-22 07:10:25,1,00:12:44,2016-04-22,22,6,Friday,2016-04-22,22,7,Friday,"40.768798828125,-73.9591293334961","40.70737075805664,-74.00443267822266"
1458642,class_one,2016-01-05 15:56:26,2016-01-05 16:02:39,1,00:06:13,2016-01-05,5,15,Tuesday,2016-01-05,5,16,Tuesday,"40.749061584472656,-73.9820785522461","40.75710678100585,-73.97463226318358"
